In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By  
import time

path = r"C:\Users\USER\Downloads\chromedriver-win64\chromedriver.exe"

options = Options()
# options.add_argument("--headless")

driver = webdriver.Chrome(path, options=options)
driver.get("https://www.psx.com.pk/market-summary/")

scroll_pause_time = 0.5  # increased pause for slower scroll
scroll_increment = 300  # smaller scroll step for smoother movement
current_position = 0
max_scroll = driver.execute_script("return document.body.scrollHeight")

while current_position < max_scroll:
    driver.execute_script(f"window.scrollTo(0, {current_position});")
    time.sleep(scroll_pause_time)
    current_position += scroll_increment
    max_scroll = driver.execute_script("return document.body.scrollHeight")


In [4]:
import pandas as pd

whole_table = driver.find_element(By.CLASS_NAME , "col-sm-12.tab-pane.inner-content-table.automobile-div.active")
tables = whole_table.find_elements(By.CLASS_NAME, "table-responsive")
tables_data = {}

for table in tables[1:]:
    table_head = table.find_element(By.TAG_NAME, "thead")
    table_body = table.find_element(By.TAG_NAME, "tbody")
    heading = table_head.text
    
    rows_data = []
    for tr in table_body.find_elements(By.TAG_NAME, "tr")[1:]:
        
        tds = tr.find_elements(By.TAG_NAME, "td")
        rows_data.append({
            "name": tds[0].text,
            "LDCP": tds[1].text,
            "OPEN": tds[2].text,
            "HIGH": tds[3].text,
            "LOW": tds[4].text,
            "CURRENT": tds[5].text,
            "CHANGE": tds[6].text,
            "VOLUME": tds[7].text
            })
    tables_data[heading] = rows_data    
    
print(tables_data)
import re

def clean_sheet_name(name):
    # Remove invalid characters and trim to 31 chars
    cleaned = re.sub(r'[:\\/*?[\]]', '', name)
    return cleaned[:31]

import pandas as pd

# Convert your dictionary to a format suitable for Excel export
excel_data = []

# Loop through each sector and its data
for sector_name, rows in tables_data.items():
    # Create a DataFrame for this sector's data
    df = pd.DataFrame(rows)
    
    # Add a column to identify the sector
    df['Sector'] = sector_name
    
    # Append to our list of DataFrames
    excel_data.append(df)

# Combine all DataFrames into one
final_df = pd.concat(excel_data, ignore_index=True)

# Alternatively, if you want each sector in a separate worksheet:
# Create a Pandas Excel writer object
with pd.ExcelWriter('psx_market_summary.xlsx', engine='xlsxwriter') as writer:
    # Write the combined data to the first sheet
    final_df.to_excel(writer, sheet_name='All Sectors', index=False)
    
    # Write each sector to its own worksheet
    for sector_name, rows in tables_data.items():
        safe_name = clean_sheet_name(sector_name)
        pd.DataFrame(rows).to_excel(writer, sheet_name=safe_name[:31], index=False)  # sheet_name limited to 31 chars

print("Excel file created successfully!")

{'AUTOMOBILE PARTS & ACCESSORIES': [{'name': 'Agriautos Ind.', 'LDCP': '110.64', 'OPEN': '111.05', 'HIGH': '113.99', 'LOW': '110.0', 'CURRENT': '111.0', 'CHANGE': '0.05', 'VOLUME': '34,705'}, {'name': 'Atlas Battery', 'LDCP': '280.13', 'OPEN': '280.99', 'HIGH': '286.0', 'LOW': '277.0', 'CURRENT': '277.01', 'CHANGE': '-2.46', 'VOLUME': '92,910'}, {'name': 'Bal.Wheels', 'LDCP': '132.16', 'OPEN': '135.02', 'HIGH': '135.02', 'LOW': '133.0', 'CURRENT': '134.0', 'CHANGE': '0.86', 'VOLUME': '2,501'}, {'name': 'Bela Automotive', 'LDCP': '135.47', 'OPEN': '136.01', 'HIGH': '140.0', 'LOW': '122.02', 'CURRENT': '132.0', 'CHANGE': '2.74', 'VOLUME': '5,712'}, {'name': 'Dewan Auto Engg', 'LDCP': '29.09', 'OPEN': '29.64', 'HIGH': '30.48', 'LOW': '28.22', 'CURRENT': '28.8', 'CHANGE': '-0.24', 'VOLUME': '2,097'}, {'name': 'Exide (PAK)', 'LDCP': '855.02', 'OPEN': '858.98', 'HIGH': '858.98', 'LOW': '818.0', 'CURRENT': '829.95', 'CHANGE': '-28.77', 'VOLUME': '16,767'}, {'name': 'Ghandhara Tyre', 'LDCP': '

In [3]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
